###### save the remove points by speed dataframe

In [1]:
import pandas as pd
import os
#import pyproj as pj # for reliable gps
# or from pyproj import Geod (and remove the pj when executing the functionality)
from pyproj import Geod
import numpy as np # for reliable gps
from collections import Counter # for reliable gps
import datetime as dt # for reliable gps

In [2]:
class TurtleData:
    """Commom base class for all turtle's data """
    
    C1 = 'Acquisition Time'
    C2 ='Acquisition Start Time'
    C3 ='Iridium CEP Radius'
    C4 ='Iridium Latitude'
    C5 ='Iridium Longitude'
    C6 ='GPS Fix Time'
    C7 ='GPS Fix Attempt'
    C8 ='GPS Latitude'
    C9 ='GPS Longitude'
    C10 ='GPS UTM Zone'
    C11 ='GPS UTM Northing'
    C12 ='GPS UTM Easting'
    C13 ='GPS Altitude'
    C14 ='GPS Horizontal Error'
    C15 ='GPS Horizontal Dilution'
    C16 ='GPS Satellite Bitmap'
    C17 ='GPS Satellite Count'
    C18 ='Underwater Percentage'
    C19 ='Dive Count'
    C20 ='Average Dive Duration'
    C21 ='Dive Duration Standard Deviation'
    C22 ='Maximum Dive Duration'
    C23 ='Maximum Dive Depth'
    C24 ='Duration Limit 1 Dive Count'
    C25 ='Duration Limit 2 Dive Count'
    C26 ='Duration Limit 3 Dive Count'
    C27 ='Duration Limit 4 Dive Count'
    C28 ='Duration Limit 5 Dive Count'
    C29 ='Duration Limit 6 Dive Count'
    C30 ='Layer 1 Percentage'
    C31 ='Layer 2 Percentage'
    C32 ='Layer 3 Percentage'
    C33 ='Layer 4 Percentage'
    C34 ='Layer 5 Percentage'
    C35 ='Layer 6 Percentage'
    C36 ='Layer 7 Percentage'
    C37 ='Layer 8 Percentage'
    C38 ='Layer 9 Percentage'
    C39 ='Layer 10 Percentage'
    C40 ='Layer 1 Dive Count'
    C41 ='Layer 2 Dive Count'
    C42 ='Layer 3 Dive Count'
    C43 ='Layer 4 Dive Count'
    C44 ='Layer 5 Dive Count'
    C45 ='Layer 6 Dive Count'
    C46 ='Layer 7 Dive Count'
    C47 ='Layer 8 Dive Count'
    C48 ='Layer 9 Dive Count'
    C49 ='Layer 10 Dive Count'
    C50 ='Temperature'
    C51 ='Satellite Uplink'
    C52 ='Receive Time'
    C53 ='Repetition Count'
    C54 ='Low Voltage'
    C55 ='Mortality'
    C56 ='Saltwater Failsafe'
    C57 ='Iridium Command'
    C58 ='Schedule Set'
    C59 ='Diagnostic Dive Data'
    C60 ='Predeployment Data'
    C61 ='Error'
    col_names = list([
        C1, C2, C3, C4, C5, C6, C7, C8, C9, C10, 
        C11, C12, C13, C14, C15, C16, C17, C18, C19, C20, 
        C21, C22, C23, C24, C25, C26, C27, C28, C29, C30, 
        C31, C32, C33, C34, C35, C36, C37, C38, C39, C40, 
        C41, C42, C43, C44, C45, C46, C47, C48, C49, C50, 
        C51, C52, C53, C54, C55, C56, C57, C58, C59, C60, 
        C61
    ])
    gps_col_names = list([
        C1, C2, C6, C7, C8, C9
    ])
    ID_ALLGPSDF_COLUMN_NAME = "All GPS's Track ID"

    @staticmethod
    def basedNamesForCsv(lastEntryRowDF, selfDfNameString, selfTurtleTag, selfSpecificFileName=""):
        for value in enumerate(lastEntryRowDF):
            #print(value[1][0])
            lastDate = value[1][0]
            date = dt.datetime.strptime(lastDate, "%Y.%m.%d")
            stringDate = date.strftime("%Y") + "_" + date.strftime("%b")
            print(f"The Last Entry in the Dataframe for {selfTurtleTag} is from: ")
            print(stringDate)
            # Give the CSV a Name based on this values above
            # name = allGpsDf_tag_xxxxx_until_lastdate
            cvsName = selfDfNameString + selfSpecificFileName + "_Tag_" + selfTurtleTag + "_" + stringDate +".csv"
            print(f"The Name for the {selfDfNameString} CSV for the turtleData {selfTurtleTag} is: ")
            print(cvsName)
            print('--------------')
            return cvsName 

    @staticmethod
    def calculateDistance(geodRef, lon1, lat1, lon2, lat2):
        # # compute forward and back azimuths, plus distance
        az12,az21,dist = geodRef.inv(lon1, lat1, lon2, lat2) #Take the second row and the first row on the count. it shoul give 3 values, but I only need the dist.
        # f"{az12:.3f} {az21:.3f} {dist:.3f}"        
        return dist #Put the dist inside the distances variable once empty.
    
    @staticmethod
    def convertUnixTimeFromString(timeString):
        return dt.datetime.strptime(timeString, '%Y.%m.%d %H:%M:%S').timestamp() #[i] is the position in an array
    
    @staticmethod
    def calculateSpeed(d, t1, t2):
        speed = d / (t2 - t1)
        return speed

    def __init__(self, tag):
        self.turtleTag = tag
        self.tagDate = ""
        self.tagTime = ""
        self.tagDatetime = ""
        self.df = pd.DataFrame()
        self.allGpsDf = pd.DataFrame()
        self.allGpsDfCsvName = ""
        #self.allGpsDf2019 = pd.DataFrame()
        self.allCleanedGpsDf = pd.DataFrame()
        self.allCleanedGpsDfCsvName = ""
        self.reliableGpsDf = pd.DataFrame()
        self.reliableGpsDfCsvName = ""
        self.noReliableGpsDf = pd.DataFrame()
        self.noReliableGpsDfCsvName = ""
    #def addElement(self, row, header):
        #self.__dict__= dict(zip(header, row))

    def addDataFromCsv(self, filename):
        temporaryDf = pd.read_csv(filename, skiprows=23, names=TurtleData.col_names)
        
        #print(f' ITS CURRENT DF IS: {self.df}') 
        #print('--------------')
        #print(f' ITS TEMPORARY DF IS {temporaryDf}') 
        
        self.df = self.df.append(temporaryDf, ignore_index=True)
        self.df.sort_values("Acquisition Time", inplace = True)

    def getTag(self):
        return self.turtleTag

    def getDf(self):        
        return self.df
    
    def giveAllGpsDf(self):
        # see all the columns in the df
        #print(self.df.columns)
        # see one column at a time        
        self.allGpsDf = self.df.copy()
        print(TurtleData.gps_col_names)
        tempList = TurtleData.gps_col_names.copy()
        for c in self.allGpsDf.columns:
            print(c)            
            if c not in tempList:
                self.allGpsDf.drop(c, inplace=True, axis=1)
            else:
                tempList.remove(c)        

        if tempList:
            print("Colummn Data missing in!")
        else:
            print("The dataframe contains all the GPS columns")
        
        print('-----DF with NaN values ---------')
        print(self.allGpsDf)       
        
        #### Eliminate those GPS's null (NaN) rows from the dataframe
        self.allGpsDf.drop(self.allGpsDf[~self.allGpsDf['GPS Latitude'].notna()].index, inplace=True)
        self.allGpsDf.reset_index(drop=True, inplace=True) # reset index

        print('-----SAME DF without NaN values ---------')
        print(self.allGpsDf)

        ####Create a column for id GPS points to the left
        trackId = self.allGpsDf.index + 1
        self.allGpsDf.insert(0, TurtleData.ID_ALLGPSDF_COLUMN_NAME, trackId)
        
        print(self.allGpsDf)        
        print(' End of all GPS Df ^')
        print('--------------')    
    
    def generateAllGpsDfCsvName(self):
        # Last entry:
        lastEntry = self.allGpsDf['Acquisition Time'].tail(1)
        #print(lastEntry)
        # separing date from time in that column
        lastEntry = pd.Series([[y for y in x.split()] for x in lastEntry])
        #print(lastEntry)
        # assign the Name in the Class Variable
        self.allGpsDfCsvName = TurtleData.basedNamesForCsv(lastEntry, "allGpsDf", self.turtleTag)        
    
    def saveAllGpsDfData(self, pathToFilePlusCsvName):
        self.allGpsDf.to_csv(pathToFilePlusCsvName, index=False)
    
    def assignTagTurtleDayDatetime(self, TagDate, TagTime):
        '''
        the Date and Time of the turtle's Tag Day
        '''
        self.tagDate = TagDate
        self.tagTime = TagTime
        self.tagDatetime = self.tagDate + " " + self.tagTime
    
    def giveAllCleanedGpsDf(self):
        # without 2019 date and without duplicate rows
        precedentYearRowsTemporaryDf = self.allGpsDf.copy()
        print(f"Before cleaning, the AllGpsDf called: {self.allGpsDfCsvName}, contained {len(precedentYearRowsTemporaryDf.index)} rows")
        
        ##### ---------- This Part is not needed ---------- #####
        # Remove 2019 data from 'Acquisition Time column:
        dateColumn = precedentYearRowsTemporaryDf['Acquisition Time']
        #print(dateColumn)
        # separing date from time in that column
        dateColumn = pd.Series([[y for y in x.split()] for x in dateColumn])
        #print(dateColumn)
        all2019DateData = []
        allOtherDateData = []        
        for value in enumerate(dateColumn):
            #print(value[1][0])
            # assign the date rows to the variable dateData
            dateData = value[1][0]
            # removing 2019 from the list
            if dateData.startswith('2019'):
                #print(f"dateData that starts with 2019 = {dateData}")
                # append the 2019 data to the list
                all2019DateData.append(dateData)
            else:
                #print(f"dateData that do not starts with 2019 = {dateData}")
                # append any other data to this list
                allOtherDateData.append(dateData)
        #print(f" 2019 list = {all2019DateData}")
        #print(f" 2020/2021 list = {allOtherDateData}")
        ##### ---------- END OF the Part not needed ---------- #####

        #### Eliminate those 2019 data rows from the dataframe
        ### example: df = df[~df['c'].astype(str).str.startswith('1')]
        print(f"Removing 2019 data from the {self.allGpsDfCsvName}")
        precedentYearRowsTemporaryDf.drop(precedentYearRowsTemporaryDf[precedentYearRowsTemporaryDf['Acquisition Time'].astype(str).str.startswith('2019')].index, inplace=True)
        precedentYearRowsTemporaryDf.reset_index(drop=True, inplace=True) # reset index
        #print(precedentYearRowsTemporaryDf)
        print(f"After removing 2019 data, the AllGpsDf called: {self.allGpsDfCsvName}, contained {len(precedentYearRowsTemporaryDf.index)} rows")
        
        ### Eliminate duplicate rows
        # Select duplicate rows except first occurrence based on all columns
        ## example of Selection by Position, to see example duplicated rows ----------------------------------
        ## df.iloc[row_indexer,column_indexer]
        print('--------------')
        duplicateRowsTemporaryDf = precedentYearRowsTemporaryDf
        duplicateRowsTemporaryDf = duplicateRowsTemporaryDf.drop_duplicates(
            [
                'Acquisition Time','Acquisition Start Time', 'GPS Fix Time', 'GPS Fix Attempt', 'GPS Latitude', 'GPS Longitude'
            ], keep='first'
        )
        print(duplicateRowsTemporaryDf)
        print(duplicateRowsTemporaryDf.iloc[13:19,1])
        print(f"Without duplicated rows, the dataframe has now {len(duplicateRowsTemporaryDf.index)} rows")
        # Drop same aquisition time that is giving us error in the calculation of distances and speeds
        duplicateRowsTemporaryDf = duplicateRowsTemporaryDf.drop_duplicates(['Acquisition Time'], keep='first')
        print(duplicateRowsTemporaryDf)
        print(duplicateRowsTemporaryDf.iloc[13:19,1])
        print("The lines where we had the same acquisition time")
        print(duplicateRowsTemporaryDf.iloc[23:29,1])
        print(f"Without duplicated acquisition times, the dataframe has now {len(duplicateRowsTemporaryDf.index)} rows")
        print("The df without duplicated rows and Without duplicated acquisition times is the duplicateRowsTemporaryDf")
        print('--------------')

        #### Eliminate test date before its Turtle tag day Datetime
        print("Excluding date BEFORE TAG DAY DATETIME")
        testDateRowsTemporaryDf = duplicateRowsTemporaryDf
        #print(testDateRowsTemporaryDf[testDateRowsTemporaryDf['Acquisition Time'].astype(str).str.startswith(self.tagDatetime)])
        ## listing days before
        print(testDateRowsTemporaryDf[testDateRowsTemporaryDf['Acquisition Time'] < self.tagDatetime])
        ## dropping them
        testDateRowsTemporaryDf.drop(testDateRowsTemporaryDf[testDateRowsTemporaryDf['Acquisition Time'] < self.tagDatetime].index, inplace=True)
        print("TEST -------------- TEST ------- TEST")
        print(self.tagDatetime)
        print(testDateRowsTemporaryDf)

        print(f"Without days before turtle tag day, the dataframe has now {len(testDateRowsTemporaryDf.index)} rows")
        print("The df without duplicated rows, without duplicated acquisition times and without days before turtle tag is the testDateRowsTemporaryDf")
        print('--------------')

        print("ALL THE DF THAT IS GONNA BE SAVE IN ALL CLEANED GPS DATAFRAME")
        print("Saving this temporary df into the allCleanedGpsDf...")
        self.allCleanedGpsDf = self.allCleanedGpsDf.append(testDateRowsTemporaryDf, ignore_index=True)
        print(self.allCleanedGpsDf)
        print(self.allCleanedGpsDf.iloc[13:19,1])
        print("The df without duplicated rows is now the self.allCleanedGpsDf")
        print('------- END -------')

    def generateAllCleanedGpsDfCsvName(self):
        # Last entry:
        lastEntry = self.allCleanedGpsDf['Acquisition Time'].tail(1)
        #print(lastEntry)
        # separing date from time in that column
        lastEntry = pd.Series([[y for y in x.split()] for x in lastEntry])
        #print(lastEntry)
        # assign the Name in the Class Variable
        self.allCleanedGpsDfCsvName = TurtleData.basedNamesForCsv(lastEntry, "allCleanedGpsDf", self.turtleTag)
    
    def saveAllCleanedGpsDfData(self, pathToFilePlusCsvName):
        self.allCleanedGpsDf.to_csv(pathToFilePlusCsvName, index=False)

    def giveReliableGpsDf(self):
        '''
        Remove GPS Errors by Angular velocity/Rotational speed 
        (degree per second)
        Geod Object for Calculations is used as objec to calculate 
        distances between points expressed in lat/lon (in degree)
        Choosing a Reference Ellipsoid - distance in degree more 
        accurate than a spherical method
        '''
        removingGpsErrorsTemporaryDf = self.allCleanedGpsDf.copy()
        #print(gpsErrorsTemporaryDf)
        wgs84_geod = Geod(ellps='WGS84')
        ## Converting data to a NumPy array.        
        latitudes = removingGpsErrorsTemporaryDf[['GPS Latitude']].to_numpy() 
        longitudes = removingGpsErrorsTemporaryDf[['GPS Longitude']].to_numpy()
        acquisitionTimes = removingGpsErrorsTemporaryDf[['Acquisition Time']].to_numpy()
        
        #latitudes = removingGpsErrorsTemporaryDf['GPS Latitude'].reset_index().values
        #longitudes = removingGpsErrorsTemporaryDf['GPS Longitude'].reset_index().values
        ##acquisitionTimes = removingGpsErrorsTemporaryDf[['Acquisition Time']].reset_index().values
        #acquisitionTimes = removingGpsErrorsTemporaryDf[['Acquisition Time']].to_numpy()        
        
        #print(latitudes.dtype)
        #print(longitudes.dtype)
        #print(acquisitionTimes.dtype)
        
        #print(latitudes)
        #print(longitudes)
        #print(acquisitionTimes)

        distances = []
        tripTimes = []
        speeds = []
        remSpeeds = []
        pointsToRemove = []        
        
        distances.append(0)
        tripTimes.append(0)
        speeds.append(0)

        i=1
        while i < (len(latitudes)):
            foundS = False
            previous = i-1
            D = 0
            S = 100
            while (S > 1.111) and (i < len(latitudes)):
                D = TurtleData.calculateDistance(wgs84_geod, longitudes[previous], latitudes[previous], longitudes[i], latitudes[i])
                t1 = TurtleData.convertUnixTimeFromString(acquisitionTimes[previous,0])
                t2 = TurtleData.convertUnixTimeFromString(acquisitionTimes[i,0])
                S = TurtleData.calculateSpeed(D,t1,t2)
                #print(f" D = {D}")
                #print('dist: %.3f' % D)
                #print(f" S = {S}")
                #print('S: %.3f' % S)
                if(S > 1.111):
                    remSpeeds.append(S)
                    #print(f"remSpeeds List: {remSpeeds}")                    
                    pointsToRemove.append(acquisitionTimes[i,0])
                    #print(pointsToRemove)                    
                    i+=1
                else:
                    foundS = True
            if(foundS):
                distances.append(D)
                tripTimes.append(t2-t1)
                speeds.append(S)
            i+=1
        print(self.turtleTag)
        print("Length of pointsToRemove List: ")
        print(len(pointsToRemove))
        print(f"remSpeeds List: {remSpeeds}")
        #---------
        print('--------------')        
        print(pointsToRemove)

        ### Cond = Points with speed > than 4km/h to be removed from the df (removingGpsErrorsTemporaryDf)
        ### Cond 2 = Points with speed > than 4km/h to be keep in the df (keepingGpsErrorsTemporaryDf)

        ### TO CREATE A DF WITHOUT SPEED ERRORS = Cond *        
        cond = removingGpsErrorsTemporaryDf['Acquisition Time'].isin(pointsToRemove)
        print('BEFORE DROP - removingGpsErrorsTemporaryDf')
        print(len(removingGpsErrorsTemporaryDf))
        removingGpsErrorsTemporaryDf.drop(removingGpsErrorsTemporaryDf[cond].index, inplace = True)
        print('AFTER DROP - removingGpsErrorsTemporaryDf')
        print(len(removingGpsErrorsTemporaryDf))        
        
        ### TO CREATE A DF JUST WITH THE SPEED ERRORS = Cond 2 *
        keepingGpsErrorsTemporaryDf = self.allCleanedGpsDf.copy()
        ##for index, rows in Time_df.head().iterrows():
         ##if(rows["Total Time"] < 6.00 ):
             ##Time_df.loc[index,"Code"] = 1
        print('BEFORE KEEPING GPS POINTS WITH SPEED > 4KM/H in the keepingGpsErrorsTemporaryDf')
        print(len(keepingGpsErrorsTemporaryDf))
        wrongSpeedPoints = []
        for indexes, rows in keepingGpsErrorsTemporaryDf.iterrows():
            if(rows['Acquisition Time'] in pointsToRemove):
                wrongSpeedPoints.append(rows)
        print(wrongSpeedPoints)
        print(len(wrongSpeedPoints))

        wrongSpeedDf = pd.DataFrame(wrongSpeedPoints)
        wrongSpeedDf.reset_index(drop=True, inplace=True)
        print("TEST AS DATAFRAME wrongSpeedPoints")
        print(wrongSpeedDf)
        print(len(wrongSpeedDf))   

        # ---------------------------------------------------
        ### CREATING NEW COLUMNS FOR BOTH DATAFRAMES AND SAVE THEM INTO A SELF DF
        ### 1 - self.reliableGpsDf
         
        removingGpsErrorsTemporaryDf['Distance (m)'] = distances        
        removingGpsErrorsTemporaryDf['Time (s)'] = tripTimes
        removingGpsErrorsTemporaryDf['Speed m/s'] = speeds
        removingGpsErrorsTemporaryDf['Time (h)'] = pd.to_timedelta(removingGpsErrorsTemporaryDf['Time (s)'], unit='s') # Add a Column with the Time passed from on Point to another in hours
        print('BEFORE CHANGES - FROM INT TO FLOAT')
        print(type(removingGpsErrorsTemporaryDf.loc[0, 'Distance (m)']))
        
        # # # Removing Square brackets From values in the 'Distance (m)' and 'Speed m/s' Columns
        # # #remove brackets of the values in Columns        
        removingGpsErrorsTemporaryDf = removingGpsErrorsTemporaryDf.astype({"Distance (m)":'float', "Speed m/s":'float'}) 
        # # #removingGpsErrorsTemporaryDf['Distance (m)'] = removingGpsErrorsTemporaryDf['Distance (m)'].str[0] #remove the brackets of the values in the column
        # # #removingGpsErrorsTemporaryDf['Speed m/s'] = removingGpsErrorsTemporaryDf['Speed m/s'].str[0] #remove the brackets of the values in the column	        
        print('AFTER CHANGES - FROM INT TO FLOAT')
        print(type(removingGpsErrorsTemporaryDf.loc[0, 'Distance (m)']))
        
        print("removingGpsErrorsTemporaryDf With new columns")
        print(removingGpsErrorsTemporaryDf)
        print(removingGpsErrorsTemporaryDf.dtypes)        
        print('--------------')
        
        # # # Create a ID Column on the Left for the Reliable Tracked Points 
        speedTrackedPoints = removingGpsErrorsTemporaryDf.index + 1
        removingGpsErrorsTemporaryDf.insert(0, 'Reliable Speed ID', speedTrackedPoints)
        
        print("removingGpsErrorsTemporaryDf With ID column")
        print(removingGpsErrorsTemporaryDf)
        print(removingGpsErrorsTemporaryDf.dtypes)        
        print('--------------')
        
        self.reliableGpsDf = self.reliableGpsDf.append(removingGpsErrorsTemporaryDf, ignore_index=True)
        
        print("Assign the Reliable GPS DF into self")
        print(self.reliableGpsDf)
        print(self.reliableGpsDf.dtypes)        
        print('--------------')

        # ---------------------------------------------------
        ### 2 - self.noReliableGpsDf
        
        wrongSpeedDf['Speeds > 1,11111 m/s'] = remSpeeds
        print('BEFORE CHANGES - FROM INT TO FLOAT')
        print(type(wrongSpeedDf.loc[0, "Speeds > 1,11111 m/s"]))
        
        # # # Removing Square brackets From values in the 'Speeds > 1,11111 m/s' Columns
        # # #remove brackets of the values in Columns        
        wrongSpeedDf = wrongSpeedDf.astype({"Speeds > 1,11111 m/s":'float'}) 
        # # #wrongSpeedDf['Speeds > 1,11111 m/s'] = wrongSpeedDf['Speeds > 1,11111 m/s'].str[0] #remove the brackets of the values in the column	        
        print('AFTER CHANGES - FROM INT TO FLOAT')
        print(type(wrongSpeedDf.loc[0, "Speeds > 1,11111 m/s"]))
        
        print("wrongSpeedDf With new columns")
        print(wrongSpeedDf)
        print(wrongSpeedDf.dtypes)        
        print('--------------')
        
        # # # Create a ID Column on the Left for the NO Reliable Tracked Points 
        speedTrackedPoints = wrongSpeedDf.index + 1
        wrongSpeedDf.insert(0, 'No Reliable Speed ID', speedTrackedPoints)
        
        print("wrongSpeedDf With ID column")
        print(wrongSpeedDf)
        print(wrongSpeedDf.dtypes)        
        print('--------------')
        
        self.noReliableGpsDf = self.noReliableGpsDf.append(wrongSpeedDf, ignore_index=True)
        
        print("Assign the NO Reliable GPS DF into self")
        print(self.noReliableGpsDf)
        print(self.noReliableGpsDf.dtypes)        
        print('--------------')
    
    def generateReliableGpsDfCsvName(self):
        # Last entry:
        lastEntry = self.reliableGpsDf['Acquisition Time'].tail(1)
        #print(lastEntry)
        # separing date from time in that column
        lastEntry = pd.Series([[y for y in x.split()] for x in lastEntry])
        #print(lastEntry)
        # assign the Name in the Class Variable
        self.reliableGpsDfCsvName = TurtleData.basedNamesForCsv(lastEntry, "reliableGpsDf", self.turtleTag, "_bySpeed")
    
    def saveReliableGpsData(self, pathToFilePlusCsvName):
        self.reliableGpsDf.to_csv(pathToFilePlusCsvName, index=False)

In [3]:
# To run with terminal OR jupyter notebook:
ASSETS_FOLDER = "assets"
ASSETS_FOLDER_ITENS = os.listdir(ASSETS_FOLDER)# ("assets")

DATACLEANINGRESULTS_FOLDER = "dataCleaningResults"
DATACLEANINGRESULTS_FOLDER_ITENS = os.listdir(DATACLEANINGRESULTS_FOLDER)# ("data_analysis/dataCleaningResults")

TAG_TURTLE_1 = '710333a'
DATE_TAG_TURTLE_1 = '2020.07.09'
TIME_TAG_TURTLE_1 = '23:00:09'

TAG_TURTLE_2 = '710348a'
DATE_TAG_TURTLE_2 = '2020.08.12'
TIME_TAG_TURTLE_2 = '02:00:11'

INITIAL_TAG_DIGITS = '7103'

# Replace spaces in filenames with underlines
def replace_space_with_underline(file_name):
    return file_name.replace(" ", "_")

# Convert excel files into csv
def converting_excel_file_into_csv_file(folder_obj, file):        
    # read excel   
    df_xlsx = pd.read_excel(os.path.join(folder_obj, file))
    # change file format
    file_in_csv = file.replace(".xlsx", ".csv")
    # transform excel to csv file with path to store the CSV file
    df_xlsx.to_csv(os.path.join(folder_obj, file_in_csv), index = False)        

# Check if some excel file has not been converted into csv yet
def check_for_excel_files():
    all_my_files = []
    n = 0
    for file in ASSETS_FOLDER_ITENS:
        # put all the file names in the same format
        file = replace_space_with_underline(file).lower()
        all_my_files.append(file)
    
    # Create a copy of list
    for file in all_my_files[:]:
        if file.endswith('.xlsx'):
            print('- Excel file = ' + file)
            file_name = file.split('.', 1)[0] # remove everything (the format) after the dot
            # remove the excel file from my all_my_files list
            all_my_files.remove(file)            
            # check if another file with the same name in the folder exists
            if any(file_name in word for word in all_my_files):            
                print(f"-- Excellent! We've already converted the excel file \'{file_name}\' into csv file")
            else:
                print(f'-- Oh No! The excel file \'{file_name}\' has been not converted. Converting it into csv file...')
                # Call function "Convert excel files into csv"
                converting_excel_file_into_csv_file(ASSETS_FOLDER, file)
                file_in_csv = file.replace(".xlsx", ".csv") 
                all_my_files.append(file_in_csv)
                print('---> ' + file_in_csv + ' has been created!')
                
    # Updated all_my_files List
    print('--- CSV files in the assets folder: ', all_my_files)

def getTurtlesData():
    split_char = '_'
    csvs = []        
    turtlesData = []
    #turtleDfs = []
    for file in ASSETS_FOLDER_ITENS:
        if file.endswith('.csv'):
            # put all the file names in the same format
            csv_string_filename = replace_space_with_underline(file).lower()
            filename_splitted = csv_string_filename.split(split_char)                        
            for word in filename_splitted:
                if word.startswith(INITIAL_TAG_DIGITS):
                    csvs.append(file)
                    currentFileCsv = ASSETS_FOLDER + '\\' + file
                    print('--------------')
                    print("Found TAG ("+ word +") in filename , check if tag is already associated with an object...")

                    #--------------------
                                
                    foundTurtleData = None
                    # check inside the list if the turtle has already been created with that tag (word)
                    for obj in turtlesData:
                        if obj.getTag() == word:
                            foundTurtleData = obj
                            break    
                    #--------------------    
                                    
                    if foundTurtleData == None:
                        print("Instance for TAG ("+ word +") NOT found! Creating a new instance...")
                        # create a TurtleData obj with the turtle tag
                        foundTurtleData = TurtleData(word)
                        turtlesData.append(foundTurtleData)
                        print("Instance for TAG ("+ word +") CREATED!")
                    else:
                        print("Instance for TAG ("+ word +") ALREADY EXISTS, skipping object creation!")
                        print('--------------')

                    # for the instances turtleData objs in the list (for each turtle tag):
                    foundTurtleData.addDataFromCsv(currentFileCsv)                    

    return turtlesData

def checkInstancesAndItsDfs(turtlesData):
    print('Created instances for Obj turtleData: ')
    for turtleData in turtlesData:        
        print(turtleData.getTag())
    print('--------------')
    print('Created Dataframes: ')
    i = 0
    for turtleData in turtlesData: 
        print(f'turtlesData[{i}].df')
        print(turtleData.turtleTag)
        print(turtleData.df)
        print('--------------')
        i+=1

def getAllGpsDataframes(turtlesData):
    for turtleData in turtlesData:
        turtleData.giveAllGpsDf()

def displayAllGpsDf(turtlesData):
    i = 0
    for turtleData in turtlesData:
        print(f'turtlesData[{i}].allGpsDf')
        print(turtleData.turtleTag)
        print(turtleData.allGpsDf)

def createAllGpsDfCsvNameForEachInstance(turtlesData):
    # create a AllGpsDf's name for each turtleData
    for turtleData in turtlesData:
        turtleData.generateAllGpsDfCsvName()

def checkIfAllGpsDfHasBeenSaved(turtlesData):
    filesInResultsFolder = []    
    
    for file in DATACLEANINGRESULTS_FOLDER_ITENS:
        filesInResultsFolder.append(file)    
    print(filesInResultsFolder)

    for turtleData in turtlesData:
        if not filesInResultsFolder:
            print(f"The filename {turtleData.allGpsDfCsvName} is not yet in the folder... saving csv")
            pathToFilePlusCsvName = os.path.join(DATACLEANINGRESULTS_FOLDER, turtleData.allGpsDfCsvName)
            turtleData.saveAllGpsDfData(pathToFilePlusCsvName)
            print(f"{turtleData.allGpsDfCsvName} has been saved in the results folder!")
            #append file in list

        elif turtleData.allGpsDfCsvName in filesInResultsFolder:
            print(f"The CSV {turtleData.allGpsDfCsvName} has already been saved in the results folder")
        else:
            print(f"The filename {turtleData.allGpsDfCsvName} is not yet in the folder... saving csv")
            pathToFilePlusCsvName = os.path.join(DATACLEANINGRESULTS_FOLDER, turtleData.allGpsDfCsvName)
            turtleData.saveAllGpsDfData(pathToFilePlusCsvName)

def assignTagDayDatetimeToEachInstance(turtlesData):
    notFoundTurtleTagDatetime = False
    for turtleData in turtlesData:
        if turtleData.getTag() == TAG_TURTLE_1:
            turtleData.assignTagTurtleDayDatetime(DATE_TAG_TURTLE_1, TIME_TAG_TURTLE_1)
        elif turtleData.getTag() == TAG_TURTLE_2:
            turtleData.assignTagTurtleDayDatetime(DATE_TAG_TURTLE_2, TIME_TAG_TURTLE_2)
        else:
            notFoundTurtleTagDatetime = True
            print("Attention!")
            print(f"{turtleData.turtleTag} has not a Tag Datetime yet!")
        # for obj in Class, print its tagDatetime
        print("For obj in Class, print its tagDatetime")
        print(turtleData.tagDatetime)
    if not notFoundTurtleTagDatetime:
        print("Tag Datetime for all instances assign!")
    
def getAllCleanedGpsDataframes(turtlesData):
    for turtleData in turtlesData:
        turtleData.giveAllCleanedGpsDf()

def createAllCleanedGpsDfCsvNameForEachInstance(turtlesData):
    # create a allCleanedGpsDf's name for each turtleData
    for turtleData in turtlesData:
        turtleData.generateAllCleanedGpsDfCsvName()

def checkIfAllCleanedGpsDfHasBeenSaved(turtlesData):
    filesInResultsFolder = []    
    
    for file in DATACLEANINGRESULTS_FOLDER_ITENS:
        filesInResultsFolder.append(file)    
    print(filesInResultsFolder)

    for turtleData in turtlesData:
        if not filesInResultsFolder:
            ## Saving AllGpsDf Data 
            #print(f"The filename {turtleData.allGpsDfCsvName} is not yet in the folder... saving csv")
            #pathToFilePlusCsvName = os.path.join(DATACLEANINGRESULTS_FOLDER, turtleData.allGpsDfCsvName)                       
            #turtleData.saveAllGpsData(pathToFilePlusCsvName)
            #print(f"{turtleData.allGpsDfCsvName} has been saved in the results folder!")            
            #print('--------------')
            ## Saving AllCleanedGps Data
            print(f"The filename {turtleData.allCleanedGpsDfCsvName} is not yet in the folder... saving csv")
            pathToFilePlusCsvName = os.path.join(DATACLEANINGRESULTS_FOLDER, turtleData.allCleanedGpsDfCsvName)
            turtleData.saveAllCleanedGpsDfData(pathToFilePlusCsvName)
            print(f"{turtleData.allCleanedGpsDfCsvName} has been saved in the results folder!")

        #elif turtleData.allGpsDfCsvName in filesInResultsFolder:
            #print(f"The CSV {turtleData.allGpsDfCsvName} has already been saved in the results folder")
        elif turtleData.allCleanedGpsDfCsvName in filesInResultsFolder:
            print(f"The CSV {turtleData.allCleanedGpsDfCsvName} has already been saved in the results folder")
        else:            
            ## Saving AllGpsDf Data
            #print(f"The filename {turtleData.allGpsDfCsvName} is not yet in the folder... saving csv")
            #pathToFilePlusCsvName = os.path.join(DATACLEANINGRESULTS_FOLDER, turtleData.allGpsDfCsvName)
            #turtleData.saveAllGpsData(pathToFilePlusCsvName)
            #print(f"{turtleData.allGpsDfCsvName} has been saved in the results folder!")              
            #print('--------------')
            ## Saving AllCleanedGps Data
            print(f"The filename {turtleData.allCleanedGpsDfCsvName} is not yet in the folder... saving csv")
            pathToFilePlusCsvName = os.path.join(DATACLEANINGRESULTS_FOLDER, turtleData.allCleanedGpsDfCsvName)
            turtleData.saveAllCleanedGpsDfData(pathToFilePlusCsvName)
            print(f"{turtleData.allCleanedGpsDfCsvName} has been saved in the results folder!")
        #print(filesInResultsFolder)
        print('--------------')

        # THIS FUNCTION ABOVE IS THE SAME FUNCTION TO SAVE THE ALL GPS DF, TRY TO DO ONLY ONE FUNCTION TO BOTH,
        # AND ALSO TRY TO MAKE THIS ONE FUNCTION TO WAIT UNTIL THE CLEANING HAS BEEN MADE TO THEN SAVE THE
        # ALL CLEANED GPS DF

def getReliableGpsDataframes(turtlesData):
    for turtleData in turtlesData:
        turtleData.giveReliableGpsDf()

def createReliableGpsDfCsvNameForEachInstance(turtlesData):
    # create a reliableGpsDf's name for each turtleData
    for turtleData in turtlesData:
        turtleData.generateReliableGpsDfCsvName()

def checkIfReliableGpsDfHasBeenSaved(turtlesData):
    filesInResultsFolder = []   
    
    for file in DATACLEANINGRESULTS_FOLDER_ITENS:
        filesInResultsFolder.append(file)    
    print(filesInResultsFolder)

    for turtleData in turtlesData:
        if not filesInResultsFolder:
            ## Saving AllGpsDf Data 
            #print(f"The filename {turtleData.allGpsDfCsvName} is not yet in the folder... saving csv")
            #pathToFilePlusCsvName = os.path.join(DATACLEANINGRESULTS_FOLDER, turtleData.allGpsDfCsvName)                       
            #turtleData.saveAllGpsData(pathToFilePlusCsvName)
            #print(f"{turtleData.allGpsDfCsvName} has been saved in the results folder!")            
            #print('--------------')
            ## Saving AllCleanedGps Data
            #print(f"The filename {turtleData.allCleanedGpsDfCsvName} is not yet in the folder... saving csv")
            #pathToFilePlusCsvName = os.path.join(DATACLEANINGRESULTS_FOLDER, turtleData.allCleanedGpsDfCsvName)
            #turtleData.saveAllCleanedGpsData(pathToFilePlusCsvName)
            #print(f"{turtleData.allCleanedGpsDfCsvName} has been saved in the results folder!")
            ## Saving ReliableGpsDf Data
            print(f"The filename {turtleData.reliableGpsDfCsvName} is not yet in the folder... saving csv")
            pathToFilePlusCsvName = os.path.join(DATACLEANINGRESULTS_FOLDER, turtleData.reliableGpsDfCsvName)
            turtleData.saveReliableGpsData(pathToFilePlusCsvName)
            print(f"{turtleData.reliableGpsDfCsvName} has been saved in the results folder!")

        #elif turtleData.allGpsDfCsvName in filesInResultsFolder:
            #print(f"The CSV {turtleData.allGpsDfCsvName} has already been saved in the results folder")
        #elif turtleData.allCleanedGpsDfCsvName in filesInResultsFolder:
            #print(f"The CSV {turtleData.allCleanedGpsDfCsvName} has already been saved in the results folder")
        elif turtleData.reliableGpsDfCsvName in filesInResultsFolder:
            print(f"The CSV {turtleData.reliableGpsDfCsvName} has already been saved in the results folder")
        else:            
            ## Saving AllGpsDf Data
            #print(f"The filename {turtleData.allGpsDfCsvName} is not yet in the folder... saving csv")
            #pathToFilePlusCsvName = os.path.join(DATACLEANINGRESULTS_FOLDER, turtleData.allGpsDfCsvName)
            #turtleData.saveAllGpsData(pathToFilePlusCsvName)
            #print(f"{turtleData.allGpsDfCsvName} has been saved in the results folder!")              
            #print('--------------')
            ## Saving AllCleanedGps Data
            #print(f"The filename {turtleData.allCleanedGpsDfCsvName} is not yet in the folder... saving csv")
            #pathToFilePlusCsvName = os.path.join(DATACLEANINGRESULTS_FOLDER, turtleData.allCleanedGpsDfCsvName)
            #turtleData.saveAllCleanedGpsData(pathToFilePlusCsvName)
            #print(f"{turtleData.allCleanedGpsDfCsvName} has been saved in the results folder!")
            ## Saving ReliableGpsDf Data
            print(f"The filename {turtleData.reliableGpsDfCsvName} is not yet in the folder... saving csv")
            pathToFilePlusCsvName = os.path.join(DATACLEANINGRESULTS_FOLDER, turtleData.reliableGpsDfCsvName)
            turtleData.saveReliableGpsData(pathToFilePlusCsvName)
            print(f"{turtleData.reliableGpsDfCsvName} has been saved in the results folder!")
        #print(filesInResultsFolder)
        print('--------------')

        # THIS FUNCTION ABOVE IS THE SAME FUNCTION TO SAVE THE ALL GPS DF AND FOR THE All CLEANED GPS, 
        # TRY TO DO ONLY ONE FUNCTION FOR ALL,
        # AND ALSO TRY TO MAKE THIS ONE FUNCTION TO WAIT UNTIL THE CLEANING HAS BEEN MADE 
        # TO THEN SAVE THE FIRST THE ALL CLEANED GPS DF AND THEN THE RELIABLE GPS DF

In [4]:
# Check if some excel file has not been converted into csv yet
check_for_excel_files()
turtlesData = getTurtlesData()

# see instances for Obj turtleData created and its dfs
checkInstancesAndItsDfs(turtlesData)
#turtlesData[0].df
#turtlesData[1].df

# build dfs of all gps
getAllGpsDataframes(turtlesData)

# see dfs of all gps
displayAllGpsDf(turtlesData)
# or
#turtlesData[0].allGpsDf
#turtlesData[1].allGpsDf

# get name for each ALL GPS DF turtleData
createAllGpsDfCsvNameForEachInstance(turtlesData)

# SAVE THE ALL GPS DATAFRAME in the Results Folder
checkIfAllGpsDfHasBeenSaved(turtlesData)

# 
assignTagDayDatetimeToEachInstance(turtlesData)

- Excel file = mytest.xlsx
-- Excellent! We've already converted the excel file 'mytest' into csv file
- Excel file = tag_710333a_20_sept.xlsx
-- Excellent! We've already converted the excel file 'tag_710333a_20_sept' into csv file
--- CSV files in the assets folder:  ['710333a_93_condensed.csv', '710348a_49_condensed.csv', 'mytest.csv', 'tag_710333a_20_sept.csv']
--------------
Found TAG (710333a) in filename , check if tag is already associated with an object...
Instance for TAG (710333a) NOT found! Creating a new instance...
Instance for TAG (710333a) CREATED!
--------------
Found TAG (710348a) in filename , check if tag is already associated with an object...
Instance for TAG (710348a) NOT found! Creating a new instance...
Instance for TAG (710348a) CREATED!
--------------
Found TAG (710333a) in filename , check if tag is already associated with an object...
Instance for TAG (710333a) ALREADY EXISTS, skipping object creation!
--------------
Created instances for Obj turtleData: 
71

Maximum Dive Depth
Duration Limit 1 Dive Count
Duration Limit 2 Dive Count
Duration Limit 3 Dive Count
Duration Limit 4 Dive Count
Duration Limit 5 Dive Count
Duration Limit 6 Dive Count
Layer 1 Percentage
Layer 2 Percentage
Layer 3 Percentage
Layer 4 Percentage
Layer 5 Percentage
Layer 6 Percentage
Layer 7 Percentage
Layer 8 Percentage
Layer 9 Percentage
Layer 10 Percentage
Layer 1 Dive Count
Layer 2 Dive Count
Layer 3 Dive Count
Layer 4 Dive Count
Layer 5 Dive Count
Layer 6 Dive Count
Layer 7 Dive Count
Layer 8 Dive Count
Layer 9 Dive Count
Layer 10 Dive Count
Temperature
Satellite Uplink
Receive Time
Repetition Count
Low Voltage
Mortality
Saltwater Failsafe
Iridium Command
Schedule Set
Diagnostic Dive Data
Predeployment Data
Error
The dataframe contains all the GPS columns
-----DF with NaN values ---------
         Acquisition Time Acquisition Start Time         GPS Fix Time  \
0     2019.07.15 14:15:00    2019.07.15 14:15:00                  NaN   
1     2019.07.15 14:15:08    2019

In [5]:
# now we need to look at the all gps df and delete the duplicates rows, before calculating the errors by speed
# deleting duplicate rows and 2019 date
getAllCleanedGpsDataframes(turtlesData)

Before cleaning, the AllGpsDf called: allGpsDf_Tag_710333a_2021_Feb.csv, contained 4652 rows
Removing 2019 data from the allGpsDf_Tag_710333a_2021_Feb.csv
After removing 2019 data, the AllGpsDf called: allGpsDf_Tag_710333a_2021_Feb.csv, contained 4641 rows
--------------
      All GPS's Track ID     Acquisition Time Acquisition Start Time  \
0                     12  2020.06.27 08:03:28    2020.06.27 08:02:40   
1                     13  2020.06.27 15:39:07    2020.06.27 15:35:59   
2                     14  2020.06.29 09:24:06    2020.06.29 09:22:54   
3                     15  2020.06.29 10:00:07    2020.06.29 10:00:00   
4                     16  2020.06.29 10:30:07    2020.06.29 10:30:00   
...                  ...                  ...                    ...   
4636                4648  2021.02.10 19:28:58    2021.02.10 19:00:00   
4637                4649  2021.02.10 22:42:32    2021.02.10 22:00:00   
4638                4650  2021.02.11 02:28:20    2021.02.11 02:00:00   
4639    

Without days before turtle tag day, the dataframe has now 2563 rows
The df without duplicated rows, without duplicated acquisition times and without days before turtle tag is the testDateRowsTemporaryDf
--------------
ALL THE DF THAT IS GONNA BE SAVE IN ALL CLEANED GPS DATAFRAME
Saving this temporary df into the allCleanedGpsDf...
      All GPS's Track ID     Acquisition Time Acquisition Start Time  \
0                     18  2020.08.12 02:00:11    2020.08.12 02:00:00   
1                     19  2020.08.12 02:47:29    2020.08.12 02:30:00   
2                     20  2020.08.12 03:02:44    2020.08.12 03:00:00   
3                     21  2020.08.12 03:33:54    2020.08.12 03:30:00   
4                     22  2020.08.12 04:20:06    2020.08.12 04:00:00   
...                  ...                  ...                    ...   
2558                2576  2021.02.10 22:28:03    2021.02.10 22:00:00   
2559                2577  2021.02.11 02:57:25    2021.02.11 02:00:00   
2560               

In [6]:
# get name for each ALL CLEANED GPS DF turtleData
createAllCleanedGpsDfCsvNameForEachInstance(turtlesData)

The Last Entry in the Dataframe for 710333a is from: 
2021_Feb
The Name for the allCleanedGpsDf CSV for the turtleData 710333a is: 
allCleanedGpsDf_Tag_710333a_2021_Feb.csv
--------------
The Last Entry in the Dataframe for 710348a is from: 
2021_Feb
The Name for the allCleanedGpsDf CSV for the turtleData 710348a is: 
allCleanedGpsDf_Tag_710348a_2021_Feb.csv
--------------


In [7]:
# SAVE THE ALL CLEANED GPS DATAFRAME in the Results Folder
checkIfAllCleanedGpsDfHasBeenSaved(turtlesData)

['allCleanedGpsDf_Tag_710333a_2021_Feb.csv', 'allCleanedGpsDf_Tag_710348a_2021_Feb.csv', 'allGpsDf_Tag_710333a_2021_Feb.csv', 'allGpsDf_Tag_710348a_2021_Feb.csv', 'reliableGpsDf_bySpeed_Tag_710333a_2021_Feb.csv', 'reliableGpsDf_bySpeed_Tag_710348a_2021_Feb.csv']
The CSV allCleanedGpsDf_Tag_710333a_2021_Feb.csv has already been saved in the results folder
--------------
The CSV allCleanedGpsDf_Tag_710348a_2021_Feb.csv has already been saved in the results folder
--------------


In [8]:
# see dfs of reliable gps (Remove GPS Errors by Angular velocity/Rotational speed)
getReliableGpsDataframes(turtlesData)

710333a
Length of pointsToRemove List: 
33
remSpeeds List: [array([1.18930457]), array([1.13096368]), array([42.3998984]), array([988.21761595]), array([282.15571977]), array([4.03555561]), array([171.75018604]), array([59.95711877]), array([10.36168531]), array([15.99201645]), array([234.99330641]), array([38.33812958]), array([218.95642372]), array([329.99169271]), array([1.40032288]), array([1.17689876]), array([151.11015997]), array([80.5629416]), array([21.39557692]), array([1.59042348]), array([27.78637835]), array([1.14580626]), array([41.69783778]), array([3.86245364]), array([1.22538072]), array([6.8776525]), array([23.15440736]), array([5.07402054]), array([9.28365653]), array([1.34491866]), array([29.8952071]), array([1.12774978]), array([111.65462206])]
--------------
['2020.07.26 05:30:07', '2020.07.28 17:33:12', '2020.08.02 04:34:30', '2020.08.05 07:01:05', '2020.08.06 08:38:31', '2020.08.06 17:36:30', '2020.08.23 19:36:02', '2020.08.24 06:58:17', '2020.08.26 01:09:49', '

710348a
Length of pointsToRemove List: 
14
remSpeeds List: [array([32.6084126]), array([1.170557]), array([177.62047653]), array([177.47599964]), array([1.55690435]), array([192.31635835]), array([168.02518142]), array([1.17419418]), array([42.03013665]), array([5.1905697]), array([10.26749834]), array([44.55296201]), array([1.25665037]), array([21.49260763])]
--------------
['2020.08.13 02:04:04', '2020.08.14 21:01:15', '2020.08.16 21:01:00', '2020.08.17 02:36:48', '2020.08.20 13:09:28', '2020.09.07 01:30:11', '2020.09.17 21:09:34', '2020.09.27 20:00:17', '2020.10.11 18:09:28', '2020.11.22 12:00:48', '2020.11.22 18:02:52', '2020.12.04 00:00:49', '2020.12.21 19:00:03', '2021.01.27 18:00:17']
BEFORE DROP - removingGpsErrorsTemporaryDf
2563
AFTER DROP - removingGpsErrorsTemporaryDf
2549
BEFORE KEEPING GPS POINTS WITH SPEED > 4KM/H in the keepingGpsErrorsTemporaryDf
2563
[All GPS's Track ID                         59
Acquisition Time          2020.08.13 02:04:04
Acquisition Start Time    

In [9]:
# get name for each RELIABLE GPS DF turtleData
createReliableGpsDfCsvNameForEachInstance(turtlesData)

The Last Entry in the Dataframe for 710333a is from: 
2021_Feb
The Name for the reliableGpsDf CSV for the turtleData 710333a is: 
reliableGpsDf_bySpeed_Tag_710333a_2021_Feb.csv
--------------
The Last Entry in the Dataframe for 710348a is from: 
2021_Feb
The Name for the reliableGpsDf CSV for the turtleData 710348a is: 
reliableGpsDf_bySpeed_Tag_710348a_2021_Feb.csv
--------------


In [10]:
# SAVE THE RELIABLE GPS DATAFRAME in the Results Folder
checkIfReliableGpsDfHasBeenSaved(turtlesData) 

['allCleanedGpsDf_Tag_710333a_2021_Feb.csv', 'allCleanedGpsDf_Tag_710348a_2021_Feb.csv', 'allGpsDf_Tag_710333a_2021_Feb.csv', 'allGpsDf_Tag_710348a_2021_Feb.csv', 'reliableGpsDf_bySpeed_Tag_710333a_2021_Feb.csv', 'reliableGpsDf_bySpeed_Tag_710348a_2021_Feb.csv']
The CSV reliableGpsDf_bySpeed_Tag_710333a_2021_Feb.csv has already been saved in the results folder
--------------
The CSV reliableGpsDf_bySpeed_Tag_710348a_2021_Feb.csv has already been saved in the results folder
--------------


##### save backup of files, rename function to giveReliableGpsDf and no Reliable gps, create function to save name for the no reliable gps and save it in the folder